In [1]:
!pip3 install yfinance
!pip3 install prophet
!pip3 install pandas

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go

In [3]:
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '${:,.2f}'.format

In [18]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2017-01-01'
ETH_df = yf.download('ETH-USD',start_date, today)
ETH_df.tail(5)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-09-29,"$1,337.55","$1,348.11","$1,293.19","$1,335.65","$1,335.65",13796915736
2022-09-30,"$1,335.65","$1,368.74","$1,320.38","$1,327.98","$1,327.98",14250100093
2022-10-01,"$1,328.19","$1,332.52","$1,306.10","$1,311.64","$1,311.64",6227961237
2022-10-02,"$1,311.75","$1,316.33","$1,275.34","$1,276.09","$1,276.09",7578351650
2022-10-03,"$1,276.16","$1,326.55","$1,271.15","$1,323.44","$1,323.44",10153070907


In [19]:
ETH_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1790 entries, 2017-11-09 to 2022-10-03
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1790 non-null   float64
 1   High       1790 non-null   float64
 2   Low        1790 non-null   float64
 3   Close      1790 non-null   float64
 4   Adj Close  1790 non-null   float64
 5   Volume     1790 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 97.9 KB


In [20]:
ETH_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [21]:
ETH_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [22]:
ETH_df.reset_index(inplace=True)
ETH_df.columns 


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [24]:
df = ETH_df[["Date", "Open"]]
new_names = {
    "Date": "ds", 
    "Open": "y",
}
df.rename(columns=new_names, inplace=True)

df.tail()

,ds,y
1785,2022-09-29,"$1,337.55"
1786,2022-09-30,"$1,335.65"
1787,2022-10-01,"$1,328.19"
1788,2022-10-02,"$1,311.75"
1789,2022-10-03,"$1,276.16"


In [25]:
# plot the open price
x = df["ds"]
y = df["y"]
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price Open Price",
)

In [26]:
m = Prophet(
    seasonality_mode="multiplicative"
)
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -48.4922
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       4331.05     0.0545278       2232.13           1           1      120   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       4541.65    0.00549555        505.27      0.7835      0.7835      238   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299        4574.2    0.00862151        319.45      0.8601      0.8601      357   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       4600.73    0.00595945       212.201      0.9059      0.9059      488   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       4606.78    0.00968601       144.536           1           1      609   
    Iter      log prob        ||dx||      ||grad||       alpha  

  4609.97     0.0063475       291.988      0.5163      0.5163      740   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     699       4617.37    0.00304595       125.284           1           1      876   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     709       4617.98    4.9199e-05       96.9865   4.335e-07       0.001      928  LS failed, Hessian reset 
     799       4618.95     0.0205923       201.128           1           1     1035   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     899       4619.94   0.000152342       72.4621      0.2682      0.2682     1159   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     999       4623.24    0.00165009       125.335           1           1     1276   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
    109

In [27]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2150,2023-09-29
2151,2023-09-30
2152,2023-10-01
2153,2023-10-02
2154,2023-10-03


In [28]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


,ds,yhat,yhat_lower,yhat_upper
2150,2023-09-29,$-570.73,"$-1,794.60",$670.03
2151,2023-09-30,$-570.88,"$-1,766.25",$624.05
2152,2023-10-01,$-578.84,"$-1,820.64",$613.75
2153,2023-10-02,$-586.77,"$-1,811.32",$669.26
2154,2023-10-03,$-593.79,"$-1,803.18",$648.64


In [14]:
#next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
#forecast[forecast['ds'] == next_day]['yhat'].item()


In [29]:
plot_plotly(m, forecast)

In [30]:
plot_components_plotly(m, forecast)

In [174]:
# Blackfriday

#Blackfriday = pd.DataFrame({
#  'holiday': 'Blackfriday',
#  'ds': pd.to_datetime(['2017-11-24','2018-11-23', '2019-11-29', '2020-11-27',
                        '2021-11-26']),
#  'lower_window': 0,
#  'upper_window': 1,
#})

#Backtoschool = pd.DataFrame({
  #'holiday': 'Backtoschool',
  #'ds': pd.to_datetime(['2017-9-1','2018-9-1', '2019-9-1', '2020-9-1',
                        '2021-9-1']),
  #'lower_window': -30,
  #'upper_window': 5,
#})



In [175]:
#holiday = pd.concat((Blackfriday, Backtoschool))

In [181]:
#m = Prophet(
   
    #holidays=holiday,
    #seasonality_prior_scale=40
   
#)
#m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -11.8209
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       3814.12     0.0129727       370.234           1           1      128   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       3910.98     0.0433595       1504.77           1           1      238   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       3944.54   0.000342204       246.721           1           1      348   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       3954.14    0.00341532       544.486      0.1263           1      457   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     418       3957.06   0.000228116       331.275   1.141e-06       0.001      515  LS failed, Hessian reset 
     499       3963.98    0.00671928    

12   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     599       3971.09     0.0154051       320.891           1           1      728   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     628       3977.48   0.000322481       135.209   1.179e-06       0.001      807  LS failed, Hessian reset 
     699       3985.56    0.00487273       94.3212           1           1      889   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     799       3988.87   0.000501204       203.085      0.2208           1     1015   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     877       3991.35    8.3344e-05       186.904   2.292e-07       0.001     1141  LS failed, Hessian reset 
     899       3992.14    0.00194771       87.3497           1           1     1167   
    Iter      log prob        ||dx||      ||grad||      

In [182]:
#future = m.make_future_dataframe(periods = 365)
#future.tail()

,ds
1755,2023-07-18
1756,2023-07-19
1757,2023-07-20
1758,2023-07-21
1759,2023-07-22


In [183]:
#forecast = m.predict(future)
#forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1755,2023-07-18,$68.02,$54.25,$80.75
1756,2023-07-19,$68.14,$54.22,$80.29
1757,2023-07-20,$68.07,$54.20,$80.57
1758,2023-07-21,$68.21,$53.85,$81.05
1759,2023-07-22,$67.41,$52.76,$80.57


In [184]:
#plot_plotly(m, forecast)

In [185]:
#plot_components_plotly(m, forecast)